In [ ]:
import requests
import json
import configparser
# from pprint import pprint

In [ ]:
config = configparser.ConfigParser()
config.read('credentials.ini')
api_user = config['GNIP_API']['username']
api_pass = config['GNIP_API']['password']

**Using GNIP API search filters**

In [ ]:
class GnipData():
    
    def __init__(maxResults, geolocation, fromDate, toDate):
        self.maxResults = maxResults
        self.geolocation = geolocation
        self.fromDate = fromDate
        self.toDate = toDate
        self._cred = (api_user, api_pass)
        self.url = "https://gnip-api.twitter.com/search/" \
        "fullarchive/accounts/greg-students/prod.json"
    
    
    def fetchTweets(queries):
        params = {'fromDate':'201401010000',
                  'toDate': '201703200000',
                  'maxResults':self.maxResults}
        
        for query in queries:
            params['query'] = query
            
            response = requests.get(self.url,
                                    params=params,
                                    auth=self._cred)

            queueKafka(response.json()['results'])
            
            while ('next' in response.json().keys()):
                params['next'] = response.json()['next']
                
                response = requests.get(url,
                                        params=params,
                                        auth=self._cred)

                queueKafka(response.json()['results'])

    
    def queueKafka(json_data):
        # queue json data in kafka
        pass


In [ ]:
class SparkProcessing():
    
    def __init__(snopes_urls):
        self.urls = snopes_urls
    
    
    def runSpark():
        getQueries(self.urls)
    
    
    def getQueries(urls):
        # spark processing on urls to get keywords
        pass

In [ ]:
# process_urls = SparkProcessing(snopes_urls)
# queries = process_urls.runSpark()

# search = GnipData(100, 'Boulder', blah, blah)
# search.fetchTweets(queries)

In [ ]:
def fetchTweets(query, maxResults=500):
    # search API url
    url = "https://gnip-api.twitter.com/search/" \
    "fullarchive/accounts/greg-students/prod.json"
    
    # storing auth credentials in a hidden variable
    _cred=(api_user, api_pass)
    
    # setting parameters to append to the url
    params = {'query':query,
              'fromDate':'201401010000',
              'toDate': '201703200000',
              'maxResults': 500}
    
    # making a GET request to the API
    response = requests.get(url,
                            params=params,
                            auth=_cred)
    
    # storing the number of tweets returned
    reslen = len(response.json()['results'])
    
    # checking if more tweets are available
    while ('next' in response.json().keys()):

        yield reslen
        
        params['next'] = response.json()['next']

        response = requests.get(url,
                                params=params,
                                auth=_cred)

        reslen += len(response.json()['results'])

In [ ]:
# using search API word locality filter
for num_tweets in fetchTweets("obama thank you"):
    print(num_tweets)


In [ ]:
import requests
import json
import configparser
from pykafka import KafkaClient

##parameters for the search
url ="https://gnip-api.twitter.com/search/fullarchive/accounts/greg-students/prod.json"
parameters = { 'query':'gnip', 'maxResults':'10',
'fromDate':'201701010000', 'toDate':'201701050000'}

##reading credentails file
config = configparser.ConfigParser()
config.read('credentials.ini')

##connecting to the kafka client
client = KafkaClient("localhost:9092")

##selcting the topic
topic = client.topics[a]

response = requests.get(url,
                        auth=(config['GNIP_API']['username'],config['GNIP_API']['password']),
                        params=parameters)

res = response.json()['results']
consumer = topic.get_simple_consumer()
##writing into topic
with topic.get_producer() as producer:
    for r in res:
        producer.produce(bytes(json.dumps(r).encode("utf-8")))
        msg = consumer.consume()
        print(msg.value)